In [0]:
!pip install scikit-learn
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 1.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('/content/drive/My Drive/Computer_vision_project/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
import scipy.stats as st
from scipy.fftpack import fft, fftfreq 
from scipy.signal import argrelextrema
import operator

In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def mean_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
    # print(sample[:,col].shape)
    average = np.mean(sample[:,col])
    feat.append(average)

  return feat

def max_ts(sample, Te=1.0):
   
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amax(sample[:,col])
      feat.append(average)

  return feat

def min_ts(sample, Te=1.0): 
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amin(sample[:,col]) # min
      feat.append(average)

  return feat 

def skew_ts(sample, Te=1.0):    
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.skew(sample[:,col]) # skew
      feat.append(average)

  return feat

def kurtosis_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.kurtosis(sample[:,col]) # kurtosis 
      feat.append(average)

  return feat
 
def iqr_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.iqr(sample[:,col]) # interquartile rante
      feat.append(average)

  return feat

def mad_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.median(np.sort(abs(sample[:,col] - np.median(sample[:,col]))))
      feat.append(average)

  return feat

def area_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col], dx=Te)
      feat.append(average)

  return feat                            
 

def sq_area_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col] ** 2, dx=Te)
      feat.append(average)

  return feat
def energy_measure(sample):
  """Calculates energy measures"""
  feat = []
  for col in range(0,sample.shape[1]):

    em_x = np.mean(np.square(sample[:,col] ))
    feat.append(em_x)
  return feat  

  # feature.append(mean_ts)
  # feature.append(max_ts)
  # feature.append(min_ts)
  # feature.append(skew_ts)
  # feature.append(kurtosis_ts)
  # feature.append(iqr_ts)
  # feature.append(mad_ts)
  # feature.append(area_ts)
  # feature.append(sq_area_ts)



In [0]:
def stat_area_features(x, Te=1.0):

    mean_ts = np.mean(x, axis=1).reshape(-1,1) # mean
    max_ts = np.amax(x, axis=1).reshape(-1,1) # max
    min_ts = np.amin(x, axis=1).reshape(-1,1) # min
    std_ts = np.std(x, axis=1).reshape(-1,1) # std
    skew_ts = st.skew(x, axis=1).reshape(-1,1) # skew
    kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1,1) # kurtosis 
    iqr_ts = st.iqr(x, axis=1).reshape(-1,1) # interquartile rante
    mad_ts = np.median(np.sort(abs(x - np.median(x, axis=1).reshape(-1,1)),
                               axis=1), axis=1).reshape(-1,1) # median absolute deviation
    area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1,1) # area under curve
    sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1,1) # area under curve ** 2

    return np.concatenate((mean_ts,max_ts,min_ts,std_ts,skew_ts,kurtosis_ts,
                           iqr_ts,mad_ts,area_ts,sq_area_ts), axis=1)
def frequency_domain_features(x, Te=1.0):
  feat=[]

  # As the DFT coefficients and their corresponding frequencies are symetrical arrays
  # with respect to the middle of the array we need to know if the number of readings 
  # in x is even or odd to then split the arrays...
  if x.shape[1]%2 == 0:
      N = int(x.shape[1]/2)
  else:
      N = int(x.shape[1]/2) - 1
  xf = np.repeat(fftfreq(x.shape[1],d=Te)[:N].reshape(1,-1), x.shape[0], axis=0) # frequencies
  dft = np.abs(fft(x, axis=1))[:,:N] # DFT coefficients   
  
  # statistical and area features
  # dft_features = stat_area_features(dft, Te=1.0)
  # weighted mean frequency
  dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1,1)
  # 5 first DFT coefficients 
  dft_first_coef = dft[:,:5]    
  # 5 first local maxima of DFT coefficients and their corresponding frequencies
  dft_max_coef = np.zeros((x.shape[0],5))
  dft_max_coef_f = np.zeros((x.shape[0],5))
  for row in range(x.shape[0]):
      # finds all local maximas indexes
      extrema_ind = argrelextrema(dft[row,:], np.greater, axis=0) 
      # makes a list of tuples (DFT_i, f_i) of all the local maxima
      # and keeps the 5 biggest...
      extrema_row = sorted([(dft[row,:][j],xf[row,j]) for j in extrema_ind[0]],
                            key=operator.itemgetter(0), reverse=True)[:5] 
      for i, ext in enumerate(extrema_row):
          dft_max_coef[row,i] = ext[0]
          dft_max_coef_f[row,i] = ext[1]  

  feat.append(np.concatenate((dft_weighted_mean_f,dft_first_coef, dft_max_coef,dft_max_coef_f), axis=1).tolist() )
  # print(len(feat))
  # print(feat)
  return feat

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks.tolist())

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    # min_feature=1000000000
    # cnt=0
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        # features = np.hstack((features, TBP(sample)))
        features = np.hstack((features, mean_ts(sample)))
        features = np.hstack((features, max_ts(sample)))
        features = np.hstack((features, min_ts(sample)))
        features = np.hstack((features, kurtosis_ts(sample)))
        features = np.hstack((features, iqr_ts(sample)))
        features = np.hstack((features, skew_ts(sample)))
        features = np.hstack((features, area_ts(sample)))
        features = np.hstack((features, sq_area_ts(sample)))
        features = np.hstack((features, mad_ts(sample)))
        # print(TBP(sample))
        # TBP_=TBP(sample)
        # for l in TBP_:
        #   features = np.hstack((features,l))
          # print('l',len(l))

          # print(l)
        # frequency_domain_features_= frequency_domain_features(sample)
        # cnt=1
        # for list_ in frequency_domain_features_:
        #   if cnt==1:
        #     for l in list_:
        #       features = np.hstack((features,l))
        #   else:
        #    features = np.hstack((features,list_))
        #   cnt+=1
               
          
          
          
        # features = np.hstack((features, energy_measure(sample)))
        # print('skew',skew_ts(sample))
        # print('kurtosis_ts',kurtosis_ts(sample))
        # print('iqr_ts',iqr_ts(sample))
        # print('A',A(sample))
        # print(len(mad_ts(sample)))
        # print(len(skew_ts(sample)))
        # print(len(ARA(sample)))
        # print(len(A(sample)))
        # print(len(SD(sample)))
        X_tmp.append(features)
        # cnt+=1
        # # print(len(features))
        # min_feature=min(min_feature,len(features))

    # print(len(X_tmp))
    X = np.array(X_tmp)
    # print(X.shape)
    return X

In [0]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc3
/device:GPU:0
Num GPUs Available:  1


In [0]:
# def Model(num_classes):
#   layers = [
    
#     #  tf.keras.layers.BatchNormalization(),
#       # tf.keras.layers.Dense(256, activation = tf.nn.relu),
#       # tf.keras.layers.LSTM(64),

#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(128, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(64, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(32, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
      
#       tf.keras.layers.Dense(num_classes, activation = 'softmax')
#   ]
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
def Model(num_classes):
  layers = [
            # tf.keras.layers.Dense(256, activation = tf.nn.leaky_relu),
            # tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Dense(128, activation =tf.nn.leaky_relu),
      # tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64, activation =tf.nn.leaky_relu),
      # tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(32, activation = tf.nn.leaky_relu),
      # tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split


In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  X_train, X_test, y_train, y_test=train_test_split( X, Y, test_size=0.33, random_state=42)
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  # train_idx = folds[0][0]
  # test_idx = folds[0][1]
  # X_train = X[train_idx]
  # X_test = X[test_idx]
  # Total_Ytrain = Y[train_idx]
  # Total_Ytest = Y[test_idx]
  # Total_Xtrain = feature_extraction(X_train)
  # Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  # for i in range(1, len(folds)):
  #         train_idx = folds[i][0]
  #         test_idx = folds[i][1]

  #         X_train = X[train_idx]
  #         X_test = X[test_idx]
  #         Y_train = Y[train_idx]
  #         Y_test = Y[test_idx]
  #         X_train = feature_extraction(X_train)
  #         X_test = feature_extraction(X_test)
  #         Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
  #         Total_Ytrain=np.append(Total_Ytrain, Y_train)
  #         Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
  #         Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, y_train.shape)
  print(X_test.shape,y_test.shape)
  # size, rows = X_train.shape
  # X_train=X_train.reshape(X_train.shape[0],rows,1)
  # y_train=y_train.reshape(y_train.shape[0],num_classes)
  print(X_train.shape)
  # _,img_rows, img_cols,_ = X.shape

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=15)
  

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, y_train, batch_size=64, epochs=200)
  _,acc=model.evaluate(X_test,y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [0]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(1711, 56) (1711,)
(844, 56) (844,)
(1711, 56)
Epoch 1/200
27/27 [==============================] - 0s 2ms/step - loss: 1.4552 - sparse_categorical_accuracy: 0.5880
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3533 - sparse_categorical_accuracy: 0.9515
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0935 - sparse_categorical_accuracy: 0.9947
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0390 - sparse_categorical_accuracy: 0.9988
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0227 - sparse_categorical_accuracy: 0.9988
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0153 - sparse_categorical_accuracy: 1.0000
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0113 - sparse_categorical_accuracy: 1.0000
Epoch 8/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0085 - spars

In [0]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(1711, 56) (1711,)
(844, 56) (844,)
(1711, 56)
Epoch 1/200
27/27 [==============================] - 0s 2ms/step - loss: 1.4157 - sparse_categorical_accuracy: 0.6201
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3121 - sparse_categorical_accuracy: 0.9673
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0815 - sparse_categorical_accuracy: 0.9965
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0361 - sparse_categorical_accuracy: 0.9994
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0223 - sparse_categorical_accuracy: 0.9988
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0148 - sparse_categorical_accuracy: 0.9994
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0104 - sparse_categorical_accuracy: 1.0000
Epoch 8/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0074 - spars

In [0]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(894, 55) (894,)
(441, 55) (441,)
(894, 55)
Epoch 1/200
14/14 [==============================] - 0s 2ms/step - loss: 1.8959 - sparse_categorical_accuracy: 0.4228
Epoch 2/200
14/14 [==============================] - 0s 2ms/step - loss: 0.8911 - sparse_categorical_accuracy: 0.8255
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 0.3984 - sparse_categorical_accuracy: 0.9407
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 0.1838 - sparse_categorical_accuracy: 0.9776
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0963 - sparse_categorical_accuracy: 0.9944
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0556 - sparse_categorical_accuracy: 0.9978
Epoch 7/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0370 - sparse_categorical_accuracy: 1.0000
Epoch 8/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0272 - sparse_c

In [0]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(1711, 56) (1711,)
(844, 56) (844,)
(1711, 56)
Epoch 1/200
27/27 [==============================] - 0s 2ms/step - loss: 1.4560 - sparse_categorical_accuracy: 0.5798
Epoch 2/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3154 - sparse_categorical_accuracy: 0.9620
Epoch 3/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0765 - sparse_categorical_accuracy: 0.9924
Epoch 4/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0350 - sparse_categorical_accuracy: 0.9982
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0198 - sparse_categorical_accuracy: 0.9988
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0132 - sparse_categorical_accuracy: 0.9994
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0094 - sparse_categorical_accuracy: 1.0000
Epoch 8/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0077 - spars

In [0]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(6582, 19) (6582,)
(3242, 19) (3242,)
(6582, 19)
Epoch 1/200
103/103 [==============================] - 0s 2ms/step - loss: 1.1487 - sparse_categorical_accuracy: 0.6402
Epoch 2/200
103/103 [==============================] - 0s 2ms/step - loss: 0.4559 - sparse_categorical_accuracy: 0.8377
Epoch 3/200
103/103 [==============================] - 0s 2ms/step - loss: 0.3646 - sparse_categorical_accuracy: 0.8666
Epoch 4/200
103/103 [==============================] - 0s 2ms/step - loss: 0.3144 - sparse_categorical_accuracy: 0.8871
Epoch 5/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2834 - sparse_categorical_accuracy: 0.8926
Epoch 6/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2532 - sparse_categorical_accuracy: 0.9037
Epoch 7/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2363 - sparse_categorical_accuracy: 0.9061
Epoch 8/200
103/103 [==============================] - 0s 2ms/step - los

In [0]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(6582, 19) (6582,)
(3242, 19) (3242,)
(6582, 19)
Epoch 1/200
103/103 [==============================] - 0s 2ms/step - loss: 1.0844 - sparse_categorical_accuracy: 0.6545
Epoch 2/200
103/103 [==============================] - 0s 2ms/step - loss: 0.4591 - sparse_categorical_accuracy: 0.8327
Epoch 3/200
103/103 [==============================] - 0s 2ms/step - loss: 0.3554 - sparse_categorical_accuracy: 0.8637
Epoch 4/200
103/103 [==============================] - 0s 2ms/step - loss: 0.3025 - sparse_categorical_accuracy: 0.8853
Epoch 5/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2669 - sparse_categorical_accuracy: 0.8967
Epoch 6/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2393 - sparse_categorical_accuracy: 0.9049
Epoch 7/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2196 - sparse_categorical_accuracy: 0.9094
Epoch 8/200
103/103 [==============================] - 0s 2ms/step - los

In [0]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(3375, 19) (3375,)
(1663, 19) (1663,)
(3375, 19)
Epoch 1/200
53/53 [==============================] - 0s 2ms/step - loss: 1.4964 - sparse_categorical_accuracy: 0.5769
Epoch 2/200
53/53 [==============================] - 0s 2ms/step - loss: 0.6290 - sparse_categorical_accuracy: 0.7917
Epoch 3/200
53/53 [==============================] - 0s 2ms/step - loss: 0.4732 - sparse_categorical_accuracy: 0.8187
Epoch 4/200
53/53 [==============================] - 0s 2ms/step - loss: 0.3921 - sparse_categorical_accuracy: 0.8563
Epoch 5/200
53/53 [==============================] - 0s 2ms/step - loss: 0.3443 - sparse_categorical_accuracy: 0.8649
Epoch 6/200
53/53 [==============================] - 0s 2ms/step - loss: 0.3139 - sparse_categorical_accuracy: 0.8791
Epoch 7/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2837 - sparse_categorical_accuracy: 0.8913
Epoch 8/200
53/53 [==============================] - 0s 2ms/step - loss: 0.2641 - spar

In [0]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(6582, 19) (6582,)
(3242, 19) (3242,)
(6582, 19)
Epoch 1/200
103/103 [==============================] - 0s 2ms/step - loss: 1.0313 - sparse_categorical_accuracy: 0.6665
Epoch 2/200
103/103 [==============================] - 0s 2ms/step - loss: 0.4388 - sparse_categorical_accuracy: 0.8364
Epoch 3/200
103/103 [==============================] - 0s 2ms/step - loss: 0.3405 - sparse_categorical_accuracy: 0.8742
Epoch 4/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2896 - sparse_categorical_accuracy: 0.8936
Epoch 5/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2571 - sparse_categorical_accuracy: 0.9032
Epoch 6/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2358 - sparse_categorical_accuracy: 0.9088
Epoch 7/200
103/103 [==============================] - 0s 2ms/step - loss: 0.2160 - sparse_categorical_accuracy: 0.9139
Epoch 8/200
103/103 [==============================] - 0s 2ms/step - los

In [0]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(2593, 11) (2593,)
(1278, 11) (1278,)
(2593, 11)
Epoch 1/200
41/41 [==============================] - 0s 2ms/step - loss: 1.7550 - sparse_categorical_accuracy: 0.4871
Epoch 2/200
41/41 [==============================] - 0s 2ms/step - loss: 1.1993 - sparse_categorical_accuracy: 0.6132
Epoch 3/200
41/41 [==============================] - 0s 2ms/step - loss: 1.0199 - sparse_categorical_accuracy: 0.6537
Epoch 4/200
41/41 [==============================] - 0s 2ms/step - loss: 0.9231 - sparse_categorical_accuracy: 0.6680
Epoch 5/200
41/41 [==============================] - 0s 2ms/step - loss: 0.8719 - sparse_categorical_accuracy: 0.6853
Epoch 6/200
41/41 [==============================] - 0s 2ms/step - loss: 0.8269 - sparse_categorical_accuracy: 0.6992
Epoch 7/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7919 - sparse_categorical_accuracy: 0.7054
Epoch 8/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7736 - spars

In [0]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(2593, 11) (2593,)
(1278, 11) (1278,)
(2593, 11)
Epoch 1/200
41/41 [==============================] - 0s 2ms/step - loss: 1.6088 - sparse_categorical_accuracy: 0.5214
Epoch 2/200
41/41 [==============================] - 0s 2ms/step - loss: 1.0924 - sparse_categorical_accuracy: 0.6302
Epoch 3/200
41/41 [==============================] - 0s 2ms/step - loss: 0.9265 - sparse_categorical_accuracy: 0.6707
Epoch 4/200
41/41 [==============================] - 0s 2ms/step - loss: 0.8480 - sparse_categorical_accuracy: 0.6857
Epoch 5/200
41/41 [==============================] - 0s 2ms/step - loss: 0.8011 - sparse_categorical_accuracy: 0.6992
Epoch 6/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7665 - sparse_categorical_accuracy: 0.7073
Epoch 7/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7445 - sparse_categorical_accuracy: 0.7173
Epoch 8/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7221 - spars

In [0]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(2599, 11) (2599,)
(1281, 11) (1281,)
(2599, 11)
Epoch 1/200
41/41 [==============================] - 0s 2ms/step - loss: 1.7548 - sparse_categorical_accuracy: 0.4575
Epoch 2/200
41/41 [==============================] - 0s 2ms/step - loss: 1.2242 - sparse_categorical_accuracy: 0.5841
Epoch 3/200
41/41 [==============================] - 0s 2ms/step - loss: 1.0418 - sparse_categorical_accuracy: 0.6333
Epoch 4/200
41/41 [==============================] - 0s 2ms/step - loss: 0.9232 - sparse_categorical_accuracy: 0.6730
Epoch 5/200
41/41 [==============================] - 0s 2ms/step - loss: 0.8489 - sparse_categorical_accuracy: 0.6899
Epoch 6/200
41/41 [==============================] - 0s 2ms/step - loss: 0.8027 - sparse_categorical_accuracy: 0.6968
Epoch 7/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7628 - sparse_categorical_accuracy: 0.7295
Epoch 8/200
41/41 [==============================] - 0s 2ms/step - loss: 0.7413 - spars

In [0]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(1437, 11) (1437,)
(709, 11) (709,)
(1437, 11)
Epoch 1/200
23/23 [==============================] - 0s 2ms/step - loss: 2.1556 - sparse_categorical_accuracy: 0.3730
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 1.4849 - sparse_categorical_accuracy: 0.5233
Epoch 3/200
23/23 [==============================] - 0s 2ms/step - loss: 1.2283 - sparse_categorical_accuracy: 0.5699
Epoch 4/200
23/23 [==============================] - 0s 2ms/step - loss: 1.0835 - sparse_categorical_accuracy: 0.6145
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 0.9835 - sparse_categorical_accuracy: 0.6541
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 0.9189 - sparse_categorical_accuracy: 0.6715
Epoch 7/200
23/23 [==============================] - 0s 2ms/step - loss: 0.8843 - sparse_categorical_accuracy: 0.6750
Epoch 8/200
23/23 [==============================] - 0s 2ms/step - loss: 0.8450 - sparse_

In [0]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(13966, 11) (13966,)
(6880, 11) (6880,)
(13966, 11)
Epoch 1/200
219/219 [==============================] - 0s 2ms/step - loss: 0.5332 - sparse_categorical_accuracy: 0.8275
Epoch 2/200
219/219 [==============================] - 0s 2ms/step - loss: 0.3019 - sparse_categorical_accuracy: 0.8947
Epoch 3/200
219/219 [==============================] - 0s 2ms/step - loss: 0.2456 - sparse_categorical_accuracy: 0.9126
Epoch 4/200
219/219 [==============================] - 0s 2ms/step - loss: 0.2135 - sparse_categorical_accuracy: 0.9258
Epoch 5/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1902 - sparse_categorical_accuracy: 0.9338
Epoch 6/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1759 - sparse_categorical_accuracy: 0.9394
Epoch 7/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1581 - sparse_categorical_accuracy: 0.9454
Epoch 8/200
219/219 [==============================] - 0s 2ms/step - lo

In [0]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(13966, 11) (13966,)
(6880, 11) (6880,)
(13966, 11)
Epoch 1/200
219/219 [==============================] - 0s 2ms/step - loss: 0.5485 - sparse_categorical_accuracy: 0.8153
Epoch 2/200
219/219 [==============================] - 0s 2ms/step - loss: 0.3041 - sparse_categorical_accuracy: 0.8952
Epoch 3/200
219/219 [==============================] - 0s 2ms/step - loss: 0.2493 - sparse_categorical_accuracy: 0.9124
Epoch 4/200
219/219 [==============================] - 0s 2ms/step - loss: 0.2157 - sparse_categorical_accuracy: 0.9226
Epoch 5/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1913 - sparse_categorical_accuracy: 0.9331
Epoch 6/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1766 - sparse_categorical_accuracy: 0.9382
Epoch 7/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1634 - sparse_categorical_accuracy: 0.9424
Epoch 8/200
219/219 [==============================] - 0s 2ms/step - lo

In [0]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(13966, 11) (13966,)
(6880, 11) (6880,)
(13966, 11)
Epoch 1/200
219/219 [==============================] - 0s 2ms/step - loss: 0.5370 - sparse_categorical_accuracy: 0.8246
Epoch 2/200
219/219 [==============================] - 0s 2ms/step - loss: 0.3049 - sparse_categorical_accuracy: 0.8968
Epoch 3/200
219/219 [==============================] - 0s 2ms/step - loss: 0.2473 - sparse_categorical_accuracy: 0.9138
Epoch 4/200
219/219 [==============================] - 0s 2ms/step - loss: 0.2110 - sparse_categorical_accuracy: 0.9271
Epoch 5/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1874 - sparse_categorical_accuracy: 0.9356
Epoch 6/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1726 - sparse_categorical_accuracy: 0.9404
Epoch 7/200
219/219 [==============================] - 0s 2ms/step - loss: 0.1561 - sparse_categorical_accuracy: 0.9459
Epoch 8/200
219/219 [==============================] - 0s 2ms/step - lo

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(7045, 11) (7045,)
(3471, 11) (3471,)
(7045, 11)
Epoch 1/200
111/111 [==============================] - 0s 2ms/step - loss: 0.6775 - sparse_categorical_accuracy: 0.7750
Epoch 2/200
111/111 [==============================] - 0s 2ms/step - loss: 0.3865 - sparse_categorical_accuracy: 0.8676
Epoch 3/200
111/111 [==============================] - 0s 2ms/step - loss: 0.3208 - sparse_categorical_accuracy: 0.8877
Epoch 4/200
111/111 [==============================] - 0s 2ms/step - loss: 0.2778 - sparse_categorical_accuracy: 0.8996
Epoch 5/200
111/111 [==============================] - 0s 2ms/step - loss: 0.2512 - sparse_categorical_accuracy: 0.9116
Epoch 6/200
111/111 [==============================] - 0s 2ms/step - loss: 0.2311 - sparse_categorical_accuracy: 0.9188
Epoch 7/200
111/111 [==============================] - 0s 2ms/step - loss: 0.2092 - sparse_categorical_accuracy: 0.9255
Epoch 8/200
111/111 [==============================] - 0s 2ms/step - loss:

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(761, 19) (761,)
(376, 19) (376,)
(761, 19)
Epoch 1/200
12/12 [==============================] - 0s 2ms/step - loss: 1.6191 - sparse_categorical_accuracy: 0.3798
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 1.0677 - sparse_categorical_accuracy: 0.6636
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 0.8364 - sparse_categorical_accuracy: 0.7057
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 0.6887 - sparse_categorical_accuracy: 0.7411
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5997 - sparse_categorical_accuracy: 0.7792
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5272 - sparse_categorical_accuracy: 0.8029
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4755 - sparse_categorical_accuracy: 0.8318
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4300 - spar

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(761, 19) (761,)
(376, 19) (376,)
(761, 19)
Epoch 1/200
12/12 [==============================] - 0s 2ms/step - loss: 1.5062 - sparse_categorical_accuracy: 0.3850
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 1.1034 - sparse_categorical_accuracy: 0.6426
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 0.8735 - sparse_categorical_accuracy: 0.7267
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 0.7108 - sparse_categorical_accuracy: 0.7569
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 0.6072 - sparse_categorical_accuracy: 0.7976
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5334 - sparse_categorical_accuracy: 0.8252
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4738 - sparse_categorical_accuracy: 0.8371
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4372 - spar

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(412, 19) (412,)
(204, 19) (204,)
(412, 19)
Epoch 1/200
7/7 [==============================] - 0s 2ms/step - loss: 1.6537 - sparse_categorical_accuracy: 0.3325
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 1.2534 - sparse_categorical_accuracy: 0.6238
Epoch 3/200
7/7 [==============================] - 0s 2ms/step - loss: 1.0867 - sparse_categorical_accuracy: 0.6869
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 0.9168 - sparse_categorical_accuracy: 0.7233
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 0.8041 - sparse_categorical_accuracy: 0.7573
Epoch 6/200
7/7 [==============================] - 0s 2ms/step - loss: 0.7111 - sparse_categorical_accuracy: 0.7791
Epoch 7/200
7/7 [==============================] - 0s 2ms/step - loss: 0.6477 - sparse_categorical_accuracy: 0.7937
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 0.5882 - sparse_categorical_a

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(761, 19) (761,)
(376, 19) (376,)
(761, 19)
Epoch 1/200
12/12 [==============================] - 0s 2ms/step - loss: 1.4724 - sparse_categorical_accuracy: 0.4179
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 1.0638 - sparse_categorical_accuracy: 0.6623
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 0.8337 - sparse_categorical_accuracy: 0.7254
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 0.6889 - sparse_categorical_accuracy: 0.7451
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5836 - sparse_categorical_accuracy: 0.7963
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 0.5070 - sparse_categorical_accuracy: 0.8173
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4498 - sparse_categorical_accuracy: 0.8515
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 0.4012 - spar

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(2526, 23) (2526,)
(1245, 23) (1245,)
(2526, 23)
Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 2.8528 - sparse_categorical_accuracy: 0.1524
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 2.2750 - sparse_categorical_accuracy: 0.3246
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 1.8782 - sparse_categorical_accuracy: 0.4390
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 1.6312 - sparse_categorical_accuracy: 0.5016
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 1.4741 - sparse_categorical_accuracy: 0.5309
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 1.3638 - sparse_categorical_accuracy: 0.5614
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 1.2783 - sparse_categorical_accuracy: 0.5998
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 1.2124 

In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(2526, 23) (2526,)
(1245, 23) (1245,)
(2526, 23)
Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 2.8254 - sparse_categorical_accuracy: 0.1683
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 2.3180 - sparse_categorical_accuracy: 0.3120
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 1.9274 - sparse_categorical_accuracy: 0.4173
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 1.6674 - sparse_categorical_accuracy: 0.4838
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 1.4982 - sparse_categorical_accuracy: 0.5241
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 1.3771 - sparse_categorical_accuracy: 0.5665
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 1.2947 - sparse_categorical_accuracy: 0.5899
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 1.2195 

In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(2526, 23) (2526,)
(1245, 23) (1245,)
(2526, 23)
Epoch 1/200
40/40 [==============================] - 0s 2ms/step - loss: 2.7956 - sparse_categorical_accuracy: 0.1675
Epoch 2/200
40/40 [==============================] - 0s 2ms/step - loss: 2.2316 - sparse_categorical_accuracy: 0.3452
Epoch 3/200
40/40 [==============================] - 0s 2ms/step - loss: 1.8298 - sparse_categorical_accuracy: 0.4446
Epoch 4/200
40/40 [==============================] - 0s 2ms/step - loss: 1.5930 - sparse_categorical_accuracy: 0.4988
Epoch 5/200
40/40 [==============================] - 0s 2ms/step - loss: 1.4565 - sparse_categorical_accuracy: 0.5432
Epoch 6/200
40/40 [==============================] - 0s 2ms/step - loss: 1.3549 - sparse_categorical_accuracy: 0.5693
Epoch 7/200
40/40 [==============================] - 0s 2ms/step - loss: 1.2787 - sparse_categorical_accuracy: 0.6025
Epoch 8/200
40/40 [==============================] - 0s 2ms/step - loss: 1.2124 

In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(1372, 22) (1372,)
(676, 22) (676,)
(1372, 22)
Epoch 1/200
22/22 [==============================] - 0s 2ms/step - loss: 2.9043 - sparse_categorical_accuracy: 0.1181
Epoch 2/200
22/22 [==============================] - 0s 2ms/step - loss: 2.5333 - sparse_categorical_accuracy: 0.2784
Epoch 3/200
22/22 [==============================] - 0s 2ms/step - loss: 2.2376 - sparse_categorical_accuracy: 0.3440
Epoch 4/200
22/22 [==============================] - 0s 2ms/step - loss: 1.9807 - sparse_categorical_accuracy: 0.4111
Epoch 5/200
22/22 [==============================] - 0s 2ms/step - loss: 1.7885 - sparse_categorical_accuracy: 0.4730
Epoch 6/200
22/22 [==============================] - 0s 2ms/step - loss: 1.6296 - sparse_categorical_accuracy: 0.5131
Epoch 7/200
22/22 [==============================] - 0s 2ms/step - loss: 1.5031 - sparse_categorical_accuracy: 0.5379
Epoch 8/200
22/22 [==============================] - 0s 2ms/step - loss: 1.4118 - 